<a href="https://colab.research.google.com/github/RKC3000/Object-Detection/blob/main/3_Day_Object_Detection_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Object Detection Challenge**

---

**YOLOv5** setup

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-140-g1db9533 Python-3.9.16 torch-2.0.0+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.4/107.7 GB disk)


**Download Dataset**

In [2]:
!bash data/scripts/get_coco.sh

######################################################################## 100.0%
######################################################################## 100.0%
######################################################################## 100.0%


In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class ob_dec(Dataset):
  

In [ ]:
imagenet_data = torchvision.datasets.ImageNet('/content/datasets/coco128/images/')
data_loader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=args.nThreads)